In [ ]:
!pip install --upgrade kneed

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from kneed import KneeLocator
from sklearn.neighbors import KernelDensity

In [ ]:
def data_cleaning(data):
  #drop na
  data.dropna(
    axis=0,
    how='any',
    subset=['Latitude','Longitude'],
    inplace=True)
  
  #define variable
  data = data.loc[:,['Latitude','Longitude']]

  return data

In [ ]:
def clustering(data):
  #define number of cluster
  kmeans_kwargs = {"init": "random",
                   "n_init": 10,
                   "random_state": 42}
  sse = []
  for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(data)
    sse.append(kmeans.inertia_)
  kl = KneeLocator(
      range(1, 20),
      sse,
      curve="convex",
      direction="decreasing"
      )
  number_of_cluster = kl.elbow.astype(np.int32)

  kmeans_tf = tf.compat.v1.estimator.experimental.KMeans(
      num_clusters = number_of_cluster
      )  

  #make a tf data
  def input_fn():
    return tf.compat.v1.train.limit_epochs(
        tf.convert_to_tensor(data.to_numpy(), dtype=tf.float32), num_epochs=1)
  
  #define centroid
  num_iterations = 10
  previous_centers = None
  for _ in range(num_iterations):
    kmeans_tf.train(input_fn)
    cluster_centers = kmeans_tf.cluster_centers()
    if previous_centers is not None:
      previous_centers = cluster_centers
  centroid = pd.DataFrame(cluster_centers, columns = ['Latitude','Longitude']) 

  #define radius
  km = KMeans(n_clusters=number_of_cluster)
  X = data.values
  Y = km.fit_predict(X)
  cluster_centroid = {}
  rad = {}
  for clust in range(len(centroid)):
    cluster_centroid[clust] = list(zip(cluster_centers[:, 0], cluster_centers[:,1]))[clust]
    rad[clust] = max([np.linalg.norm(np.subtract(i, cluster_centroid[clust]))for i in zip(X[Y == clust, 0], X[Y == clust, 1])])  
  rad = [float(q) for q in list(rad.values())]
  radius=np.array(rad)
  radius = pd.DataFrame(radius, columns=['Radius'])

  #make a dataframe for output
  output = pd.concat([centroid, radius], axis=1)
  
  return output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data=pd.read_csv('/content/drive/Shareddrives/CapstoneProject/Datasets/train.csv')

In [ ]:
data = data_cleaning(data)

In [ ]:
coba = clustering(data)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpveu25j9p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

In [ ]:
coba

,Latitude,Longitude,Radius
0,41.715584,-87.612770,0.202485
1,41.840130,-87.701340,0.258046
2,41.965790,-87.849159,0.249054
3,41.966259,-87.712807,0.353065
